# Headers

In [13]:

import pandas as pd 
from collections import defaultdict
import igraph as ig



# Class for Session Node to store time of visit for User -> POI 


In [6]:

class SessionNode:
    def __init__(self, hourSlot):
        self.hour = hourSlot
        


# Class for User Node to store user info


In [7]:

class UserNode:
    def __init__(self, userId):
        self.userId = userId



# Class for Point of Interest (POI) Node to store various POIs like malls, parks, restaurants, visiting places etc..


In [8]:

class POINode:
    def __init__(self, venueId, venueTypeId=None, venueName=None, latitude=None, longitude=None):
        self.venueId = venueId
        self.venueTypeId = venueTypeId
        self.venueName = venueName
        self.location = (latitude, longitude)
        


# Class for graph that represents POIs, user info and time of visits


In [9]:

class Graph:
    def __init__(self):
        self.userToSessionLinks = defaultdict(list) # userNode  -> [(sessionNode, weight)]
        self.sessionToPOILinks = defaultdict(list)  # sessionNode -> [(POINode, weight)]
        
        #auxiliary lookups for handling redundant data from dataset
        self.userIdToUserNodes = {}
        self.venueIdToPOINOdes = {}
    
    
    def loadGraph(self,dataset):
        data = pd.read_csv(dataset) 
        for index, row in data.head(5).iterrows():
            #print(row['userID'], row['venueID'])
            #userID,venueID,date,year,monthOfYear,dayOfWeek,dayOfMonth,dayHour,visitCount
            userNode = None
            poiNode = None
            sessionNode = None
            
            userId = row['userID']
            if userId in self.userIdToUserNodes:
                userNode = self.userIdToUserNodes[userId]
            else:
                userNode = UserNode(userId) 
                self.userIdToUserNodes[userId] = userNode
            
            venueId = row['venueID']
            if venueId in self.venueIdToPOINOdes:
                poiNode = self.venueIdToPOINOdes[venueId]
            else:
                poiNode = POINode(venueId)
                self.venueIdToPOINOdes[venueId] = poiNode
            
            hourSlot = row['dayHour']
            
            if userNode in self.userToSessionLinks:
                sessionNodeWeights = self.userToSessionLinks[userNode]
                #find hour slot in existing sessionNodes
                foundHourSlot = False
                for sNode in sessionNodeWeights:
                    #import pdb; pdb.set_trace()
                    sessionNode = sNode[0]
                    if sessionNode.hour == hourSlot:
                        sNode[1] += 1    #incrementing weight for the hour slot 
                        foundHourSlot = True
                        break
                if not foundHourSlot:
                    sessionNode = SessionNode(hourSlot)
                    self.userToSessionLinks[userNode].append([sessionNode, 1])
            
            else:
                #no userNode mapping
                sessionNode = SessionNode(hourSlot)
                self.userToSessionLinks[userNode] = [[sessionNode, 1]]
            
            
            if sessionNode in self.sessionToPOILinks:
                poiNodeWeights = self.sessionToPOILinks[sessionNode]
                foundPOINode = False
                for pNode in poiNodeWeights:
                    if pNode[0] == poiNode:
                        pNode[1] += 1
                        foundPOINode = True
                        break
                if not foundPOINode:
                    self.sessionToPOILinks[sessionNode].append([poiNode, 1])
            else:
                self.sessionToPOILinks[sessionNode] = [[poiNode, 1]]
            
    
    
    def lookupFromGraph(self, userId):
        pass
        
    

In [22]:
graph = Graph()
graph.loadGraph("export.csv")
print(graph.userIdToUserNodes)
print ('----------------------')
print(graph.userToSessionLinks)
print ('----------------------')
print (graph.venueIdToPOINOdes)
print ('----------------------')
print (graph.sessionToPOILinks)

{7509: <__main__.UserNode instance at 0x7f4dde899fc8>}
----------------------
defaultdict(<type 'list'>, {<__main__.UserNode instance at 0x7f4dde899fc8>: [[<__main__.SessionNode instance at 0x7f4ddf20fd88>, 1], [<__main__.SessionNode instance at 0x7f4dd8430950>, 1], [<__main__.SessionNode instance at 0x7f4dd8430830>, 1], [<__main__.SessionNode instance at 0x7f4dd8430878>, 1], [<__main__.SessionNode instance at 0x7f4dd9c6e098>, 1]]})
----------------------
{34034: <__main__.POINode instance at 0x7f4dd84303f8>, 487163: <__main__.POINode instance at 0x7f4dd8430170>, 11492: <__main__.POINode instance at 0x7f4dde8b9518>, 47021: <__main__.POINode instance at 0x7f4dd84308c0>, 28838: <__main__.POINode instance at 0x7f4dd8a1c830>}
----------------------
defaultdict(<type 'list'>, {<__main__.SessionNode instance at 0x7f4ddf20fd88>: [[<__main__.POINode instance at 0x7f4dd8a1c830>, 1]], <__main__.SessionNode instance at 0x7f4dd9c6e098>: [[<__main__.POINode instance at 0x7f4dd84308c0>, 1]], <__main

In [21]:

def visualizeGraph():
    edges = []
    for key in graph.userToSessionLinks:
        userNode = key
        sessionNodesList = graph.userToSessionLinks[key]
        for item in sessionNodesList:
            edges.append(tuple([userNode, item[0]]))
    print (edges)
    G = ig.Graph(edges, directed=False)
    